In [15]:
%load_ext autoreload
%autoreload 1

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [16]:
import pandas as pd
import numpy as np
import os
os.chdir("/home/yangkai/codegen-detection")

In [17]:
gen_data = pd.read_pickle("dataset/apps_gen_test100.pkl")
hum_data = pd.read_pickle("dataset/apps_hum_test100.pkl")
all_data = pd.concat([gen_data,hum_data])

In [18]:
all_identifier_map = all_data.identifier_map.tolist()
all_full_func = all_data.extracted_full_func.tolist()

In [19]:
import random
random.seed(42)
all_replaced_full_code = []

replaced_prob = 0.1

for ident_map,full_func in zip(all_identifier_map,all_full_func):
    offset = 0
    new_name_dict = {var_name:f"var{i}" if random.random() < replaced_prob else var_name for i,var_name in enumerate(ident_map['var'])}
    flat_name_loc = []
    for var_name in ident_map['var']:
        for loc in ident_map['var'][var_name]:
            flat_name_loc.append((var_name,loc))
    flat_name_loc.sort(key=lambda x:x[1],reverse=True)

    offset = 0

    for orig_name,start_loc in flat_name_loc:
        # print(orig_name,offset)
        new_name = new_name_dict[orig_name]
        full_func = full_func[:start_loc+offset] + new_name + full_func[start_loc+len(orig_name)+offset:]
        # offset += len(new_name) - len(orig_name)

    all_replaced_full_code.append(full_func)

In [20]:
all_data['ident_replaced_prob01'] = all_replaced_full_code

/tmp/ipykernel_1524931/1582002190.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  all_data['ident_replaced_prob01'] = all_replaced_full_code


In [21]:
gen_data = all_data[all_data['label'] == 1]
hum_data = all_data[all_data['label'] == 0]

In [23]:
hum_data

,problem_id,question,solutions,difficulty,url,starter_code,apps_split,code_source,solution,extracted_full_func,...,ident_replaced_prob02_word_prob_llama-13b,ident_replaced_prob02_word_rank_llama-13b,ident_replaced_prob02_entropy_llama-13b,ident_replaced_prob02_word_prob_incoder-6B,ident_replaced_prob02_word_rank_incoder-6B,ident_replaced_prob02_entropy_incoder-6B,ident_replaced_prob02_word_prob_codegen-2B-mono,ident_replaced_prob02_word_rank_codegen-2B-mono,ident_replaced_prob02_entropy_codegen-2B-mono,ident_replaced_prob01
2341,2341,You and your $n - 1$ friends have found an arr...,[import sys\nreadline = sys.stdin.readline\n\n...,competition,https://codeforces.com/problemset/problem/1290/A,,train,codeforces.com,import sys\nreadline = sys.stdin.readline\n\nc...,import sys\nreadline = sys.stdin.readline\ncla...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,import sys\nreadline = sys.stdin.readline\ncla...
3226,3226,Task: Given an array arr of strings complete t...,"[def land_perimeter(arr):\n \n I,J = len(...",introductory,https://www.codewars.com/kata/5839c48f0cf94640...,\ndef land_perimeter(arr):\n\t,train,www.codewars.com,def land_perimeter(arr):\n total=0\n for x i...,def land_perimeter(arr):\n total=0\n for x i...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,def land_perimeter(arr):\n var1=0\n for x in...
7273,7273,You have a simple undirected graph consisting ...,[import sys\ninput = lambda: sys.stdin.readlin...,interview,https://codeforces.com/problemset/problem/1228/D,,test,codeforces.com,"n,m=list(map(int,input().split()))\nEE=[]\nif ...","n,m=list(map(int,input().split()))\nEE=[]\nif ...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"n,m=list(map(int,input().split()))\nvar2=[]\ni..."
4554,4554,# Letterss of Natac\n\nIn a game I just made u...,[from collections import Counter\n\ndef play_i...,introductory,https://www.codewars.com/kata/59e0069781618a79...,"\ndef play_if_enough(hand, play):\n\t",train,www.codewars.com,from collections import Counter\n\ndef play_if...,from collections import Counter\ndef play_if_e...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,from collections import Counter\ndef play_if_e...
738,738,Calculate the power of an army of numbers from...,[# cook your dish here\n# cook your dish here\...,interview,https://www.codechef.com/FULU2020/problems/ARMY1N,,train,www.codechef.com,# cook your dish here\n#powerful numbers\nn = ...,"n = int(input())\nplist = [2, 3, 5, 7, 11, 13,...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"n = int(input())\nplist = [2, 3, 5, 7, 11, 13,..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1607,1607,"""QAQ"" is a word to denote an expression of cry...",[s=input()\nans = 0\nfor i in range(len(s)):\n...,interview,https://codeforces.com/problemset/problem/894/A,,train,codeforces.com,# IAWT\ns = input()\nans = 0\nfor i in range(l...,s = input()\nans = 0\nfor i in range(len(s)):\...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,var0 = input()\nans = 0\nfor i in range(len(va...
4413,4413,## Task\n\nComplete function `splitOddAndEven`...,[def split_odd_and_even(n):\n \n import ...,introductory,https://www.codewars.com/kata/57a2ab1abb994466...,\ndef split_odd_and_even(n):\n\t,train,www.codewars.com,from itertools import groupby\n\ndef split_odd...,from itertools import groupby\ndef split_odd_a...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,from itertools import groupby\ndef split_odd_a...
5632,5632,"Orac is studying number theory, and he is inte...",[import sys\ninput = sys.stdin.readline\n\ndef...,interview,https://codeforces.com/problemset/problem/1350/A,,test,codeforces.com,"t=int(input())\nfor i in range(t):\n n,k=map(...","t=int(input())\nfor i in range(t):\n n,k=map(...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"t=int(input())\nfor i in range(t):\n n,k=map(..."
3517,3517,# Let's watch a parade!\n\n## Brief\n\nYou're ...,"[def parade_time(groups, location, speed, pref...",introductory,https://www.codewars.com/kata/560d41fd7e0b946a...,"\ndef parade_time(groups, location,

In [24]:
hum_data.to_pickle("dataset/apps_hum_test100.pkl")
gen_data.to_pickle("dataset/apps_gen_test100.pkl")

In [22]:
all_data['ident_replaced_code'] = all_replaced_full_code

In [7]:
import functools
import io


@functools.total_ordering
class TextPosition:
    """
    By using total_ordering, only __lt__ and __eq__ have to be defined in order to make objects of type TextPosition
    comparable.
    Lines and columns start counting with 1.
    """

    def __init__(self, line, column):
        assert line > 0, f"Text lines must be positive numbers, not `{line}`!"
        assert column > 0, f"Text columns must be positive numbers, not `{column}`!"
        self.line = line
        self.column = column

    @staticmethod
    def from_ast_pos(ast_pos):
        return TextPosition(ast_pos['line'], ast_pos['column'])
    @staticmethod
    def from_tree_sitter_pos(pos):
        return TextPosition(pos[0]+1,pos[1]+1)

    def __lt__(self, other):
        if self.line == other.line:
            return self.column < other.column
        return self.line < other.line

    def __eq__(self, other):
        return self.line == other.line and self.column == other.column

    def __str__(self):
        return f"{self.line},{self.column}"


class RangeInterval:
    """
    Right-open integer interval representing spans in text.
    """

    def __init__(self, start_pos, end_pos):
        assert end_pos >= start_pos, f"An interval cannot have a negative range! {start_pos} - {end_pos}"
        self.start_pos = start_pos
        self.end_pos = end_pos

    @staticmethod
    def from_semantic(span):
        return RangeInterval(TextPosition.from_ast_pos(span['start']), TextPosition.from_ast_pos(span['end']))

    @staticmethod
    def from_java_parser(span):
        return RangeInterval(TextPosition.from_ast_pos(span['begin']), TextPosition.from_ast_pos(span['end']))

    @staticmethod
    def from_tree_sitter(start,end):
        return RangeInterval(TextPosition.from_tree_sitter_pos(start),TextPosition.from_tree_sitter_pos(end))

    @staticmethod
    def from_python_token(span):
        start_line, start_col = span.start
        end_line, end_col = span.end
        return RangeInterval(TextPosition(start_line, start_col + 1), TextPosition(end_line, end_col + 1))

    @staticmethod
    def from_compressed(compressed_interval):
        return RangeInterval(TextPosition(compressed_interval[0][0], compressed_interval[0][1]),
                             TextPosition(compressed_interval[1][0], compressed_interval[1][1]))

    @staticmethod
    def empty_interval():
        return RangeInterval(TextPosition(1, 1), TextPosition(1, 1))

    def compress(self):
        return ((self.start_pos.line, self.start_pos.column), (self.end_pos.line, self.end_pos.column))

    def contains(self, other):
        return self.start_pos <= other.start_pos and self.end_pos >= other.end_pos

    def intersects(self, other):
        return self.start_pos < other.end_pos and self.end_pos > other.start_pos

    def substring(self, text):
        lines = io.StringIO(text).readlines()
        lines = [line for i, line in enumerate(lines) if self.start_pos.line <= i + 1 <= self.end_pos.line]
        substring = []
        for i, line in enumerate(lines):
            idx_start = 0
            idx_end = len(line)
            if i == 0:
                idx_start = self.start_pos.column - 1
            if i == len(lines) - 1:
                idx_end = self.end_pos.column - 1
            substring.append(line[idx_start:idx_end])

        return ''.join(substring)

    def is_smaller_than(self, other):
        line_span_self = self.end_pos.line - self.start_pos.line
        line_span_other = other.end_pos.line - other.start_pos.line
        if line_span_self < line_span_other:
            return True
        elif line_span_self > line_span_other:
            return False
        else:
            col_span_self = self.end_pos.column - self.start_pos.column
            col_span_other = other.end_pos.column - other.start_pos.column
            return col_span_self - col_span_other < 0

    def __str__(self):
        return f"[{self.start_pos} - {self.end_pos}]"

In [8]:
import pygments
import io
class PygmentsTokenizer:

    def __init__(self, language):
        self.language = language
        self._lexer = pygments.lexers.get_lexer_by_name(language)

    def tokenize(self, code: str):
        lines = io.StringIO(code).readlines()
        line_lengths = {i + 1: len(line) for i, line in enumerate(lines)}

        # When using multiprocessing with joblib it sometimes can happen that the lexer is not properly instantiated
        # In this case, we just create a new object
        if not hasattr(self._lexer, "_tokens"):
            self._lexer = pygments.lexers.get_lexer_by_name(self.language)
        unprocessed_tokens = self._lexer.get_tokens_unprocessed(code)

        current_line = 1
        current_line_start_pos = 0
        tokens = []
        # Calculate source spans
        for pos, token_type, string in unprocessed_tokens:
            if str(token_type) == "Token.Error":
                continue
            if current_line_start_pos + line_lengths[current_line] <= pos:
                current_line_start_pos += line_lengths[current_line]
                current_line += 1
            start_line = current_line
            start_col = pos - current_line_start_pos + 1
            end_line = current_line
            multiline_pos = current_line_start_pos
            while multiline_pos + line_lengths[end_line] < pos + len(string):
                multiline_pos += line_lengths[end_line]
                end_line += 1
            current_line = end_line
            current_line_start_pos = multiline_pos
            end_col = pos - multiline_pos + len(string) + 1
            source_span = RangeInterval(TextPosition(start_line, start_col), TextPosition(end_line, end_col))
            tokens.append(Token(string, source_span, token_type))

        return tokens


class Token:
    def __init__(self, string, source_span, token_type):
        self.string = string
        self.source_span = source_span
        self.token_type = token_type

    def __str__(self):
        return f"{self.source_span} ({self.token_type}) '{self.string}'"


class CommentsRemover:

    def __init__(self, tokenizer):
        self.tokenizer = tokenizer

    def filter(self, code):
        tokens = self.tokenizer.tokenize(code)
        last_token_comment = None
        for t in tokens:
            if last_token_comment:
                if t.string == '\n':
                    last_token_comment.string = ''
                else:
                    last_token_comment.string = '\n'
                last_token_comment = None
            if t.token_type in pygments.token.Comment.Multiline:
                t.string = ' '
            elif t.token_type in pygments.token.Literal.String.Doc:
                t.string = ' '
            # elif t.token_type in pygments.token.Comment and str(t.token_type) != "Token.Comment.Preproc":
            elif t.token_type in pygments.token.Comment:
                last_token_comment = t
        return ''.join([t.string for t in tokens])

class EmptyLinesRemover:
    def filter(self, code):
        lines = io.StringIO(code).readlines()
        stripped_lines = []
        last_line_empty = False
        # for line in lines:
        #     if last_line_empty:
        #         if not line.isspace():
        #             last_line_empty = False
        #             stripped_lines.append(line)
        #     else:
        #         if line.isspace():
        #             last_line_empty = True
        #         stripped_lines.append(line)
        for line in lines:
            if not line.isspace():
                stripped_lines.append(line)
        return ''.join(stripped_lines)

In [23]:
tokenizer = PygmentsTokenizer('Python')
coment_remover = CommentsRemover(tokenizer)
empty_remover = EmptyLinesRemover()
def func_extract(code):
    code = coment_remover.filter(code)
    return empty_remover.filter(code)

In [24]:
all_data['extracted_full_func'] = all_data.full_code.apply(func_extract)

In [25]:
all_data['ident_replaced_extracted_full_func'] = all_data.ident_replaced_code.apply(func_extract)

In [26]:
gen_data = all_data[all_data.label == 1]
hum_data = all_data[all_data.label == 0]

In [27]:
len(gen_data),len(hum_data)

(10758, 8226)

In [28]:
gen_data.to_pickle("dataset/all_gen_test_prompt_split.pkl")
hum_data.to_pickle("dataset/all_human_test_prompt_split.pkl")